In [1]:
from hyperopt import fmin, tpe, hp, partial
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, zero_one_loss

import xgboost as xgb
import pandas as pd

def GetNewDataByPandas():
    wine = pd.read_csv("/home/fonttian/Data/UCI/wine/wine.csv")
    wine['alcohol**2'] = pow(wine["alcohol"], 2)
    wine['volatileAcidity*alcohol'] = wine["alcohol"] * wine['volatile acidity']
    y = np.array(wine.quality)
    X = np.array(wine.drop("quality", axis=1))
    # X = np.array(wine)

    columns = np.array(wine.columns)

    return X, y, columns

In [2]:
# Read wine quality data from file
X, y, wineNames = GetNewDataByPandas()

# split data to [[0.8,0.2],01]
x_train_all, x_predict, y_train_all, y_predict = train_test_split(X, y, test_size=0.10, random_state=100)

x_train, x_test, y_train, y_test = train_test_split(x_train_all, y_train_all, test_size=0.2, random_state=100)

In [3]:
def argsDict_tranform(argsDict, isPrint=False):
    argsDict["max_depth"] = argsDict["max_depth"] + 5
    argsDict['n_estimators'] = argsDict['n_estimators'] + 150
    argsDict["learning_rate"] = argsDict["learning_rate"] * 0.02 + 0.05
    argsDict["subsample"] = argsDict["subsample"] * 0.1 + 0.5
    argsDict["colsample_bytree"] = argsDict["colsample_bytree"] * 0.1 + 0.5
    argsDict["min_child_weight"] = argsDict["min_child_weight"] + 1
    argsDict["gamma"] = argsDict["gamma"] * 0.1
    if isPrint:
        print(argsDict)
    else:
        pass

    return argsDict

In [14]:
def xgbRegressor_factory(argsDict):
    argsDict = argsDict_tranform(argsDict)

    xrf = xgb.XGBRegressor(nthread=-1,  # 进程数
                           max_depth=argsDict['max_depth'],  # 最大深度
                           n_estimators=argsDict['n_estimators'],  # 树的数量
                           learning_rate=argsDict['learning_rate'],  # 学习率
                           subsample=argsDict['subsample'],  # 采样数
                           min_child_weight=argsDict['min_child_weight'],  # 孩子数
                           max_delta_step=10,  # 10步不降则停止
                           objective='reg:linear',
                           silent=0,  # 是否显示
                           gamma=argsDict['gamma'],  # 是否后剪枝
                           colsample_bytree=argsDict['colsample_bytree'],  # 样本列采样
                           colsample_bylevel=1,
                           reg_alpha=0,  # L1 正则化
                           reg_lambda=1,  # L2 正则化
                           scale_pos_weight=1,  # 取值>0时,在数据不平衡时有助于收敛
                           seed=619,  # 随机种子
                           missing=None
                           )

    return get_tranformer_score(xrf)

In [11]:
def get_tranformer_score(tranformer):
    xrf = tranformer
    
    # without eval_set

    # xlf.fit(x_train_all, y_train_all, eval_metric='rmse', verbose=True)
    # prediction = xlf.predict(x_predict)

    # with eval_set
    xrf.fit(x_train, y_train, eval_metric='rmse', verbose=True, eval_set=[(x_train, y_train), (x_test, y_test)],
            early_stopping_rounds=300)
    prediction = xrf.predict(x_predict, ntree_limit=xrf.best_ntree_limit)
  
    return mean_squared_error(y_predict, prediction)

In [12]:
# 自定义hyperopt的参数空间
space = {"max_depth": hp.randint("max_depth", 15),
         "n_estimators": hp.randint("n_estimators", 300),
         "learning_rate": hp.randint("learning_rate", 6),
         "subsample": hp.randint("subsample", 5),
         "colsample_bytree": hp.randint('colsample_bytree', 5),
         "min_child_weight": hp.randint("min_child_weight", 6),
         "gamma":hp.randint("gamma",6),
         }

In [13]:
# 开始使用hyperopt进行自动调参
algo = partial(tpe.suggest, n_startup_jobs=1)
best = fmin(xgbRegressor_factory, space, algo=algo, max_evals=20, pass_expr_memo_ctrl=None)

[0]	validation_0-rmse:4.71819	validation_1-rmse:4.84972
Multiple eval metrics have been passed: 'validation_1-rmse' will be used for early stopping.

Will train until validation_1-rmse hasn't improved in 300 rounds.
[1]	validation_0-rmse:4.30534	validation_1-rmse:4.43298
[2]	validation_0-rmse:3.93291	validation_1-rmse:4.05578
[3]	validation_0-rmse:3.59325	validation_1-rmse:3.71597
[4]	validation_0-rmse:3.28343	validation_1-rmse:3.40295
[5]	validation_0-rmse:3.00344	validation_1-rmse:3.1232
[6]	validation_0-rmse:2.74935	validation_1-rmse:2.86698
[7]	validation_0-rmse:2.5186	validation_1-rmse:2.63502
[8]	validation_0-rmse:2.3095	validation_1-rmse:2.42344
[9]	validation_0-rmse:2.11784	validation_1-rmse:2.23127
[10]	validation_0-rmse:1.94307	validation_1-rmse:2.05355
[11]	validation_0-rmse:1.7892	validation_1-rmse:1.90092
[12]	validation_0-rmse:1.6481	validation_1-rmse:1.75956
[13]	validation_0-rmse:1.52279	validation_1-rmse:1.63493
[14]	validation_0-rmse:1.41042	validation_1-rmse:1.52061


[138]	validation_0-rmse:0.03916	validation_1-rmse:0.582003
[139]	validation_0-rmse:0.038589	validation_1-rmse:0.58181
[140]	validation_0-rmse:0.037695	validation_1-rmse:0.581761
[141]	validation_0-rmse:0.036779	validation_1-rmse:0.581744
[142]	validation_0-rmse:0.035989	validation_1-rmse:0.581875
[143]	validation_0-rmse:0.035234	validation_1-rmse:0.581993
[144]	validation_0-rmse:0.034521	validation_1-rmse:0.582193
[145]	validation_0-rmse:0.033765	validation_1-rmse:0.582287
[146]	validation_0-rmse:0.033108	validation_1-rmse:0.5823
[147]	validation_0-rmse:0.032434	validation_1-rmse:0.58244
[148]	validation_0-rmse:0.031742	validation_1-rmse:0.58234
[149]	validation_0-rmse:0.031139	validation_1-rmse:0.582248
[150]	validation_0-rmse:0.030258	validation_1-rmse:0.582239
[151]	validation_0-rmse:0.029511	validation_1-rmse:0.582441
[152]	validation_0-rmse:0.02868	validation_1-rmse:0.582457
[153]	validation_0-rmse:0.027994	validation_1-rmse:0.582447
[154]	validation_0-rmse:0.027336	validation_1-r

[35]	validation_0-rmse:0.412842	validation_1-rmse:0.658005
[36]	validation_0-rmse:0.397395	validation_1-rmse:0.647255
[37]	validation_0-rmse:0.381887	validation_1-rmse:0.640359
[38]	validation_0-rmse:0.367794	validation_1-rmse:0.632602
[39]	validation_0-rmse:0.35463	validation_1-rmse:0.625366
[40]	validation_0-rmse:0.342635	validation_1-rmse:0.622097
[41]	validation_0-rmse:0.329798	validation_1-rmse:0.618924
[42]	validation_0-rmse:0.323393	validation_1-rmse:0.6175
[43]	validation_0-rmse:0.312449	validation_1-rmse:0.613186
[44]	validation_0-rmse:0.303892	validation_1-rmse:0.610081
[45]	validation_0-rmse:0.294211	validation_1-rmse:0.607905
[46]	validation_0-rmse:0.287106	validation_1-rmse:0.605967
[47]	validation_0-rmse:0.28224	validation_1-rmse:0.605346
[48]	validation_0-rmse:0.2749	validation_1-rmse:0.605458
[49]	validation_0-rmse:0.267626	validation_1-rmse:0.603196
[50]	validation_0-rmse:0.262916	validation_1-rmse:0.602346
[51]	validation_0-rmse:0.256285	validation_1-rmse:0.599929
[52

[174]	validation_0-rmse:0.018392	validation_1-rmse:0.582188
[175]	validation_0-rmse:0.018109	validation_1-rmse:0.582211
[176]	validation_0-rmse:0.01793	validation_1-rmse:0.582187
[177]	validation_0-rmse:0.017714	validation_1-rmse:0.582242
[178]	validation_0-rmse:0.017357	validation_1-rmse:0.582349
[179]	validation_0-rmse:0.017155	validation_1-rmse:0.582301
[180]	validation_0-rmse:0.016815	validation_1-rmse:0.582367
[181]	validation_0-rmse:0.016421	validation_1-rmse:0.582237
[182]	validation_0-rmse:0.016185	validation_1-rmse:0.582217
[183]	validation_0-rmse:0.01588	validation_1-rmse:0.582186
[184]	validation_0-rmse:0.01554	validation_1-rmse:0.582196
[185]	validation_0-rmse:0.015357	validation_1-rmse:0.582267
[186]	validation_0-rmse:0.015043	validation_1-rmse:0.582307
[187]	validation_0-rmse:0.014795	validation_1-rmse:0.582343
[188]	validation_0-rmse:0.014521	validation_1-rmse:0.582306
[189]	validation_0-rmse:0.014301	validation_1-rmse:0.582378
[190]	validation_0-rmse:0.014025	validation

[37]	validation_0-rmse:0.483713	validation_1-rmse:0.610393
[38]	validation_0-rmse:0.481211	validation_1-rmse:0.609897
[39]	validation_0-rmse:0.478267	validation_1-rmse:0.607962
[40]	validation_0-rmse:0.473656	validation_1-rmse:0.609429
[41]	validation_0-rmse:0.468763	validation_1-rmse:0.606163
[42]	validation_0-rmse:0.467166	validation_1-rmse:0.607633
[43]	validation_0-rmse:0.461446	validation_1-rmse:0.607652
[44]	validation_0-rmse:0.459053	validation_1-rmse:0.607746
[45]	validation_0-rmse:0.454359	validation_1-rmse:0.607103
[46]	validation_0-rmse:0.449675	validation_1-rmse:0.606366
[47]	validation_0-rmse:0.44689	validation_1-rmse:0.60835
[48]	validation_0-rmse:0.444625	validation_1-rmse:0.610205
[49]	validation_0-rmse:0.440598	validation_1-rmse:0.609713
[50]	validation_0-rmse:0.437125	validation_1-rmse:0.6102
[51]	validation_0-rmse:0.433032	validation_1-rmse:0.608822
[52]	validation_0-rmse:0.430863	validation_1-rmse:0.608754
[53]	validation_0-rmse:0.42748	validation_1-rmse:0.607581
[5

...

Multiple eval metrics have been passed: 'validation_1-rmse' will be used for early stopping.

Will train until validation_1-rmse hasn't improved in 300 rounds.
[1]	validation_0-rmse:4.3021	validation_1-rmse:4.43171
[2]	validation_0-rmse:3.92736	validation_1-rmse:4.05471
[3]	validation_0-rmse:3.58637	validation_1-rmse:3.71032
[4]	validation_0-rmse:3.27549	validation_1-rmse:3.39564
[5]	validation_0-rmse:2.99441	validation_1-rmse:3.11696
[6]	validation_0-rmse:2.73962	validation_1-rmse:2.86044
[7]	validation_0-rmse:2.50942	validation_1-rmse:2.63007
[8]	validation_0-rmse:2.29865	validation_1-rmse:2.41589
[9]	validation_0-rmse:2.10731	validation_1-rmse:2.22465
[10]	validation_0-rmse:1.93356	validation_1-rmse:2.0504
[11]	validation_0-rmse:1.77608	validation_1-rmse:1.89472
[12]	validation_0-rmse:1.63465	validation_1-rmse:1.75118
[13]	validation_0-rmse:1.50583	validation_1-rmse:1.62847
[14]	validation_0-rmse:1.38964	validation_1-rmse:1.51203
[15]	validation_0-rmse:1.2832	validation_1-rmse:1.413

[138]	validation_0-rmse:0.001626	validation_1-rmse:0.5884
[139]	validation_0-rmse:0.001557	validation_1-rmse:0.588386
[140]	validation_0-rmse:0.001492	validation_1-rmse:0.588385
[141]	validation_0-rmse:0.001439	validation_1-rmse:0.58839
[142]	validation_0-rmse:0.001376	validation_1-rmse:0.588395
[143]	validation_0-rmse:0.001318	validation_1-rmse:0.588407
[144]	validation_0-rmse:0.001261	validation_1-rmse:0.588407
[145]	validation_0-rmse:0.00121	validation_1-rmse:0.588405
[146]	validation_0-rmse:0.001164	validation_1-rmse:0.588404
[147]	validation_0-rmse:0.001119	validation_1-rmse:0.58841
[148]	validation_0-rmse:0.001087	validation_1-rmse:0.588403
[149]	validation_0-rmse:0.001049	validation_1-rmse:0.588403
[150]	validation_0-rmse:0.00101	validation_1-rmse:0.588404
[151]	validation_0-rmse:0.00097	validation_1-rmse:0.588399
[152]	validation_0-rmse:0.00094	validation_1-rmse:0.588398
[153]	validation_0-rmse:0.000902	validation_1-rmse:0.5884
[154]	validation_0-rmse:0.000881	validation_1-rmse

[275]	validation_0-rmse:0.000616	validation_1-rmse:0.588413
[276]	validation_0-rmse:0.000616	validation_1-rmse:0.588413
[277]	validation_0-rmse:0.000616	validation_1-rmse:0.588413
[278]	validation_0-rmse:0.000616	validation_1-rmse:0.588413
[279]	validation_0-rmse:0.000616	validation_1-rmse:0.588413
[280]	validation_0-rmse:0.000616	validation_1-rmse:0.588413
[281]	validation_0-rmse:0.000616	validation_1-rmse:0.588413
[282]	validation_0-rmse:0.000616	validation_1-rmse:0.588413
[283]	validation_0-rmse:0.000616	validation_1-rmse:0.588413
[284]	validation_0-rmse:0.000616	validation_1-rmse:0.588413
[285]	validation_0-rmse:0.000616	validation_1-rmse:0.588413
[286]	validation_0-rmse:0.000616	validation_1-rmse:0.588413
[287]	validation_0-rmse:0.000616	validation_1-rmse:0.588413
[288]	validation_0-rmse:0.000616	validation_1-rmse:0.588413
[289]	validation_0-rmse:0.000616	validation_1-rmse:0.588413
[290]	validation_0-rmse:0.000616	validation_1-rmse:0.588413
[291]	validation_0-rmse:0.000613	validat

[58]	validation_0-rmse:0.079299	validation_1-rmse:0.596557
[59]	validation_0-rmse:0.07486	validation_1-rmse:0.595548
[60]	validation_0-rmse:0.070743	validation_1-rmse:0.595062
[61]	validation_0-rmse:0.067067	validation_1-rmse:0.594779
[62]	validation_0-rmse:0.063275	validation_1-rmse:0.594399
[63]	validation_0-rmse:0.059867	validation_1-rmse:0.593939
[64]	validation_0-rmse:0.056619	validation_1-rmse:0.593475
[65]	validation_0-rmse:0.053679	validation_1-rmse:0.59295
[66]	validation_0-rmse:0.050893	validation_1-rmse:0.592523
[67]	validation_0-rmse:0.0482	validation_1-rmse:0.592199
[68]	validation_0-rmse:0.045896	validation_1-rmse:0.591909
[69]	validation_0-rmse:0.043544	validation_1-rmse:0.591799
[70]	validation_0-rmse:0.041343	validation_1-rmse:0.591625
[71]	validation_0-rmse:0.039188	validation_1-rmse:0.591906
[72]	validation_0-rmse:0.037149	validation_1-rmse:0.591823
[73]	validation_0-rmse:0.035218	validation_1-rmse:0.591713
[74]	validation_0-rmse:0.033353	validation_1-rmse:0.591849
[

[196]	validation_0-rmse:0.00064	validation_1-rmse:0.589811
[197]	validation_0-rmse:0.00064	validation_1-rmse:0.589811
[198]	validation_0-rmse:0.00064	validation_1-rmse:0.58981
[199]	validation_0-rmse:0.00064	validation_1-rmse:0.58981
[200]	validation_0-rmse:0.000633	validation_1-rmse:0.589811
[201]	validation_0-rmse:0.000633	validation_1-rmse:0.589811
[202]	validation_0-rmse:0.000633	validation_1-rmse:0.58981
[203]	validation_0-rmse:0.000633	validation_1-rmse:0.58981
[204]	validation_0-rmse:0.000633	validation_1-rmse:0.58981
[205]	validation_0-rmse:0.000633	validation_1-rmse:0.58981
[206]	validation_0-rmse:0.000633	validation_1-rmse:0.58981
[207]	validation_0-rmse:0.000629	validation_1-rmse:0.58981
[208]	validation_0-rmse:0.000629	validation_1-rmse:0.589809
[209]	validation_0-rmse:0.000629	validation_1-rmse:0.58981
[210]	validation_0-rmse:0.000629	validation_1-rmse:0.58981
[211]	validation_0-rmse:0.000629	validation_1-rmse:0.58981
[212]	validation_0-rmse:0.000629	validation_1-rmse:0.58

In [15]:
# 展示结果
RMSE = xgbRegressor_factory(best)
# print('mse of the best xgboost:', RMSE)
print('best :', best)
print('best param after transform :')
argsDict_tranform(best,isPrint=True)
print('rmse of the best xgboost:', np.sqrt(RMSE))

[14:07:55] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=3
[0]	validation_0-rmse:4.71567	validation_1-rmse:4.8463
Multiple eval metrics have been passed: 'validation_1-rmse' will be used for early stopping.

Will train until validation_1-rmse hasn't improved in 300 rounds.
[14:07:55] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[1]	validation_0-rmse:4.3021	validation_1-rmse:4.43171
[14:07:55] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[2]	validation_0-rmse:3.92736	validation_1-rmse:4.05471
[14:07:55] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 0 pruned nodes, max_depth=4
[3]	validation_0-rmse:3.58637	validation_1-rmse:3.71032
[14:07:55] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 0 pruned nodes, max_depth=5
[4]	va

[14:07:57] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 676 extra nodes, 0 pruned nodes, max_depth=17
[45]	validation_0-rmse:0.166747	validation_1-rmse:0.611156
[14:07:58] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 744 extra nodes, 0 pruned nodes, max_depth=17
[46]	validation_0-rmse:0.157298	validation_1-rmse:0.609076
[14:07:58] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 660 extra nodes, 0 pruned nodes, max_depth=17
[47]	validation_0-rmse:0.148586	validation_1-rmse:0.607724
[14:07:58] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 702 extra nodes, 0 pruned nodes, max_depth=17
[48]	validation_0-rmse:0.140176	validation_1-rmse:0.607103
[14:07:58] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 860 extra nodes, 0 pruned nodes, max_depth=17
[49]	validation_0-rmse:0.131754	validation_1-rmse:0.605376
[14:07:58] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 7

[14:07:59] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 958 extra nodes, 0 pruned nodes, max_depth=17
[90]	validation_0-rmse:0.015052	validation_1-rmse:0.590603
[14:07:59] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 686 extra nodes, 0 pruned nodes, max_depth=17
[91]	validation_0-rmse:0.014331	validation_1-rmse:0.590619
[14:07:59] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 962 extra nodes, 0 pruned nodes, max_depth=17
[92]	validation_0-rmse:0.01364	validation_1-rmse:0.590609
[14:07:59] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 930 extra nodes, 0 pruned nodes, max_depth=17
[93]	validation_0-rmse:0.013036	validation_1-rmse:0.590588
[14:07:59] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 922 extra nodes, 0 pruned nodes, max_depth=17
[94]	validation_0-rmse:0.012424	validation_1-rmse:0.590518
[14:07:59] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 79

[14:08:00] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 388 extra nodes, 0 pruned nodes, max_depth=17
[135]	validation_0-rmse:0.001973	validation_1-rmse:0.58987
[14:08:00] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 432 extra nodes, 0 pruned nodes, max_depth=17
[136]	validation_0-rmse:0.001884	validation_1-rmse:0.589862
[14:08:00] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 430 extra nodes, 0 pruned nodes, max_depth=17
[137]	validation_0-rmse:0.001802	validation_1-rmse:0.589859
[14:08:00] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 324 extra nodes, 0 pruned nodes, max_depth=17
[138]	validation_0-rmse:0.001743	validation_1-rmse:0.589861
[14:08:00] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 404 extra nodes, 0 pruned nodes, max_depth=17
[139]	validation_0-rmse:0.001666	validation_1-rmse:0.589852
[14:08:00] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 root

[14:08:00] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[180]	validation_0-rmse:0.000642	validation_1-rmse:0.589812
[14:08:00] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[181]	validation_0-rmse:0.000642	validation_1-rmse:0.589812
[14:08:00] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[182]	validation_0-rmse:0.000642	validation_1-rmse:0.589812
[14:08:00] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 4 extra nodes, 0 pruned nodes, max_depth=2
[183]	validation_0-rmse:0.000642	validation_1-rmse:0.589812
[14:08:00] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[184]	validation_0-rmse:0.000642	validation_1-rmse:0.589811
[14:08:00] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 4 extra nod

[14:08:01] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[226]	validation_0-rmse:0.000625	validation_1-rmse:0.58981
[14:08:01] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[227]	validation_0-rmse:0.000625	validation_1-rmse:0.58981
[14:08:01] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[228]	validation_0-rmse:0.000625	validation_1-rmse:0.58981
[14:08:01] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[229]	validation_0-rmse:0.000625	validation_1-rmse:0.58981
[14:08:01] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[230]	validation_0-rmse:0.000625	validation_1-rmse:0.58981
[14:08:01] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0

[271]	validation_0-rmse:0.000618	validation_1-rmse:0.589811
[14:08:02] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[272]	validation_0-rmse:0.000618	validation_1-rmse:0.589811
[14:08:02] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=3
[273]	validation_0-rmse:0.000617	validation_1-rmse:0.58981
[14:08:02] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[274]	validation_0-rmse:0.000617	validation_1-rmse:0.58981
[14:08:02] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[275]	validation_0-rmse:0.000617	validation_1-rmse:0.589811
[14:08:02] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[276]	validation_0-rmse:0.000617	validation_1-rmse:0.58981
[14:08:02] /workspace/src/tree/up

"\nbest : {'colsample_bytree': 0.7, 'gamma': 0.0, 'learning_rate': 0.15000000000000002, 'max_depth': 16, 'min_child_weight': 1, 'n_estimators': 440, 'subsample': 0.8}\nbest param after transform :\n{'colsample_bytree': 0.57, 'gamma': 0.0, 'learning_rate': 0.053000000000000005, 'max_depth': 21, 'min_child_weight': 2, 'n_estimators': 590, 'subsample': 0.5800000000000001}\nrmse of the best xgboost: 0.5602862515362897\n"

In [16]:
"""
best : {'colsample_bytree': 0.7, 'gamma': 0.0, 'learning_rate': 0.15000000000000002, 'max_depth': 16, 'min_child_weight': 1, 'n_estimators': 440, 'subsample': 0.8}
best param after transform :
{'colsample_bytree': 0.57, 'gamma': 0.0, 'learning_rate': 0.053000000000000005, 'max_depth': 21, 'min_child_weight': 2, 'n_estimators': 590, 'subsample': 0.5800000000000001}
rmse of the best xgboost: 0.5602862515362897
"""

"\nbest : {'colsample_bytree': 0.7, 'gamma': 0.0, 'learning_rate': 0.15000000000000002, 'max_depth': 16, 'min_child_weight': 1, 'n_estimators': 440, 'subsample': 0.8}\nbest param after transform :\n{'colsample_bytree': 0.57, 'gamma': 0.0, 'learning_rate': 0.053000000000000005, 'max_depth': 21, 'min_child_weight': 2, 'n_estimators': 590, 'subsample': 0.5800000000000001}\nrmse of the best xgboost: 0.5602862515362897\n"